In [5]:
import networkx as nx
import pandas as pd
import numpy as np
import random

In [6]:
def create_dummy_topology_data(num_rings=10, nodes_per_ring=10, logical_rings_per_physical=3):
    """
    Create dummy topology data for testing
    
    Args:
        num_rings: Number of physical rings to create
        nodes_per_ring: Number of nodes per ring
        logical_rings_per_physical: Number of logical rings per physical ring
        
    Returns:
        DataFrame with topology data
    """
    data = []
    
    for pr_idx in range(num_rings):
        physical_ring = f"RING_{pr_idx}"
        block_name = f"BLOCK_{pr_idx}"  # One block per physical ring
        
        # Each physical ring has multiple logical rings from the same block
        for lr_idx in range(logical_rings_per_physical):
            logical_ring = f"LR_{pr_idx}_{lr_idx}"
            
            # Create a linear path for each logical ring (not a complete ring)
            for i in range(nodes_per_ring - 1):  # Connect nodes in a path, not a ring
                node_a = f"NODE_{pr_idx}_{lr_idx}_{i}"
                node_b = f"NODE_{pr_idx}_{lr_idx}_{i+1}"
                
                data.append({
                    'aendname': node_a,
                    'bendname': node_b,
                    'aendip': f"10.{pr_idx}.{lr_idx}.{i}",
                    'bendip': f"10.{pr_idx}.{lr_idx}.{i+1}",
                    'aendifIndex': i,
                    'bendifIndex': i+1,
                    'block_name': block_name,
                    'physicalringname': physical_ring,
                    'lrname': logical_ring
                })
            
            # Connect the first and last nodes to the block
            # First node connects to block
            data.append({
                'aendname': f"NODE_{pr_idx}_{lr_idx}_0",
                'bendname': block_name,
                'aendip': f"10.{pr_idx}.{lr_idx}.0",
                'bendip': f"10.{pr_idx}.99.99",  # Special IP for block
                'aendifIndex': 100 + lr_idx,
                'bendifIndex': 100 + lr_idx,
                'block_name': block_name,
                'physicalringname': physical_ring,
                'lrname': logical_ring
            })
            
            # Last node connects to block
            data.append({
                'aendname': f"NODE_{pr_idx}_{lr_idx}_{nodes_per_ring-1}",
                'bendname': block_name,
                'aendip': f"10.{pr_idx}.{lr_idx}.{nodes_per_ring-1}",
                'bendip': f"10.{pr_idx}.99.99",  # Special IP for block
                'aendifIndex': 200 + lr_idx,
                'bendifIndex': 200 + lr_idx,
                'block_name': block_name,
                'physicalringname': physical_ring,
                'lrname': logical_ring
            })
    
    # Add connections between blocks from different physical rings
    # for pr_idx in range(num_rings):
    #     if pr_idx < num_rings - 1:  # Connect to next physical ring
    #         # Connect this block to the next ring's block
    #         block_a = f"BLOCK_{pr_idx}"
    #         block_b = f"BLOCK_{pr_idx+1}"
            
    #         data.append({
    #             'aendname': block_a,
    #             'bendname': block_b,
    #             'aendip': f"10.{pr_idx}.99.99",
    #             'bendip': f"10.{pr_idx+1}.99.99",
    #             'aendifIndex': 300 + pr_idx,
    #             'bendifIndex': 300 + pr_idx + 1,
    #             'block_name': block_a,
    #             'physicalringname': f"RING_{pr_idx}",
    #             'lrname': "INTER_BLOCK"  # Inter-block connection
    #         })
    
    return pd.DataFrame(data)

In [3]:
topo_data = create_dummy_topology_data()


In [7]:
def build_graph_with_position_features(topology_df):
    """Build NetworkX graph with enhanced position features"""
    G = nx.Graph()
    
    # Track ring membership and positions
    ring_positions = {}  # (pr_id, lr_id) -> list of positions
    
    # First pass: identify all rings and node positions
    for _, row in topology_df.iterrows():
        for node_col in ['aendname', 'bendname']:
            node = row[node_col]
            if not isinstance(node, str) or 'NODE_' not in node:
                continue
                
            parts = node.split('_')
            if len(parts) >= 4:
                try:
                    pr_id = int(parts[1])
                    lr_id = int(parts[2])
                    pos = int(parts[3])
                    
                    key = (pr_id, lr_id)
                    if key not in ring_positions:
                        ring_positions[key] = []
                    
                    if pos not in ring_positions[key]:
                        ring_positions[key].append(pos)
                except ValueError:
                    continue
    
    # Sort positions within each ring
    for key in ring_positions:
        ring_positions[key].sort()
    
    # Add nodes and edges with position-aware features
    for _, row in topology_df.iterrows():
        aend = row['aendname']
        bend = row['bendname']
        
        # Add nodes with enhanced features
        for node in [aend, bend]:
            if node in G:
                continue  # Skip if already added
                
            # Default features
            features = {
                'is_block': 'BLOCK' in str(node),
                'pr_id': -1,
                'lr_id': -1,
                'position': -1,
  
            }
            
            # Extract position information
            if isinstance(node, str) and 'NODE_' in node:
                parts = node.split('_')
                if len(parts) >= 4:
                    try:
                        pr_id = int(parts[1])
                        lr_id = int(parts[2])
                        pos = int(parts[3])
                        
                        # Get normalized position (crucial for learning the pattern)


                        
                        features.update({
                            'pr_id': pr_id,
                            'lr_id': lr_id,
                            'position': pos,

                        })
                    except ValueError:
                        pass
            
            G.add_node(node, **features)
        
        # Add edge
        G.add_edge(aend, bend)
    
    return G

In [5]:
G = build_graph_with_position_features(topo_data)

In [6]:
def add_failure_status_to_graph(G, failed_node_names=None):
    """Add 'failed' attribute to graph nodes"""
    if failed_node_names is None:
        failed_node_names = []
    
    # Create a copy to avoid modifying the original
    G_copy = G.copy()
    
    # Set failed attribute for all nodes
    for node in G_copy.nodes():
        G_copy.nodes[node]['failed'] = node in failed_node_names
    
    return G_copy

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data, DataLoader
import networkx as nx
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [8]:
class GraphStateGNN(nn.Module):
    def __init__(self, in_channels):
        super(GraphStateGNN, self).__init__()
        
        # Feature processing
        self.node_encoder = nn.Linear(in_channels, 64)
        
        # Graph convolution layers
        self.conv1 = GCNConv(64, 128)
        self.conv2 = GCNConv(128, 64)
        
        # Isolation prediction layer
        self.predictor = nn.Linear(64, 1)
    
    def forward(self, x, edge_index):
        """The first feature of x is the 'failed' status"""
        # Initial node encoding
        h = F.relu(self.node_encoder(x))
        
        # Message passing to understand graph structure
        h = F.relu(self.conv1(h, edge_index))
        h = F.dropout(h, p=0.2, training=self.training)
        
        h = F.relu(self.conv2(h, edge_index))
        
        # Predict isolation probability
        out = torch.sigmoid(self.predictor(h))
        
        # Make sure failed nodes are never predicted as isolated
        # failed status is the first feature (x[:, 0])
        failed_mask = (x[:, 0] < 0.5).float().unsqueeze(1)
        out = out * failed_mask
        
        return out

In [9]:
def calculate_isolated_nodes(G, node_list, node_to_idx):
    """Calculate which nodes are isolated based on graph failures"""
    y = torch.zeros(len(node_list), dtype=torch.float)
    
    for i, node in enumerate(node_list):
        # Skip failed nodes
        if G.nodes[node].get('failed', False):
            continue
            
        # Get node attributes
        attrs = G.nodes[node]
        pr_id = attrs.get('pr_id', -1)
        lr_id = attrs.get('lr_id', -1)
        pos = attrs.get('position', -1)
        
        if pr_id < 0 or lr_id < 0 or pos < 0:
            continue
        
        # Find failed nodes in same ring
        failed_in_ring = []
        for other_node in node_list:
            if not G.nodes[other_node].get('failed', False):
                continue
                
            other_attrs = G.nodes[other_node]
            other_pr = other_attrs.get('pr_id', -1)
            other_lr = other_attrs.get('lr_id', -1)
            other_pos = other_attrs.get('position', -1)
            
            # Check if in same ring
            if other_pr == pr_id and other_lr == lr_id and other_pos >= 0:
                failed_in_ring.append((other_node, other_pos))
        
        # Check if between any two failed nodes
        if len(failed_in_ring) >= 2:
            for i in range(len(failed_in_ring)):
                for j in range(i+1, len(failed_in_ring)):
                    pos1 = failed_in_ring[i][1]
                    pos2 = failed_in_ring[j][1]
                    
                    if min(pos1, pos2) < pos < max(pos1, pos2):
                        y[node_to_idx[node]] = 1.0
                        break
    
    return y

In [10]:
def prepare_data_from_graph(G):
    """Convert graph with failure attributes to PyG data"""
    from torch_geometric.data import Data
    
    # Create node mapping
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Create edge index
    edge_index = []
    for u, v in G.edges():
        edge_index.append([node_to_idx[u], node_to_idx[v]])
        edge_index.append([node_to_idx[v], node_to_idx[u]])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t()
    
    # Create node features
    x = []
    failed_nodes = []
    for i, node in enumerate(node_list):
        attrs = G.nodes[node]
        
        # Get failed status as first feature
        is_failed = float(attrs.get('failed', False))
        if is_failed > 0.5:
            failed_nodes.append(i)
        
        features = [
            is_failed,                       # Failed status as first feature
            float(attrs.get('is_block', False)),
            attrs.get('pr_id', -1) / 10.0,
            attrs.get('lr_id', -1) / 10.0,
            attrs.get('position', -1) / 10.0,
        ]
        x.append(features)
    
    x = torch.tensor(x, dtype=torch.float)
    
    # Calculate isolated nodes (ground truth)
    y = calculate_isolated_nodes(G, node_list, node_to_idx)
    
    return Data(x=x, edge_index=edge_index, y=y)

In [11]:
def train_graph_state_model(graph_examples, num_epochs=30):
    """Train model on graphs with failed attributes"""
    from torch_geometric.loader import DataLoader
    from sklearn.metrics import precision_score, recall_score, f1_score
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Prepare training data
    train_data = [prepare_data_from_graph(G) for G in graph_examples]
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    
    # Create model
    in_channels = train_data[0].x.size(1)
    model = GraphStateGNN(in_channels).to(device)
    
    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Class weights
    pos_weight = torch.tensor([10.0]).to(device)
    
    for epoch in range(1, num_epochs + 1):
        # Training
        model.train()
        total_loss = 0
        all_preds, all_labels = [], []
        
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            out = model(batch.x, batch.edge_index)
            
            # Loss with weighting
            loss = F.binary_cross_entropy(
                out.squeeze(), 
                batch.y,
                weight=pos_weight * batch.y + (1.0 - batch.y)
            )
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            # Track predictions
            pred = (out.squeeze() > 0.5).float()
            all_preds.append(pred.detach().cpu())
            all_labels.append(batch.y.detach().cpu())
        
        # Calculate metrics
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        precision = precision_score(all_labels, all_preds, zero_division=0)
        recall = recall_score(all_labels, all_preds, zero_division=0)
        f1 = f1_score(all_labels, all_preds, zero_division=0)
        
        print(f"Epoch {epoch}/{num_epochs}:")
        print(f"  Loss: {total_loss/len(train_loader):.4f}")
        print(f"  Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        print(f"  Predictions: {all_preds.sum().item()}/{len(all_preds)}")
    
    return model

In [16]:
def generate_graph_examples(G, num_examples=100):
    """Generate graphs with different failure patterns"""
    import random
    
    examples = []
    nodes = list(G.nodes())
    
    for _ in range(num_examples):
        # Randomly select 1-4 failed nodes
        num_failures = 2
        failed_nodes = random.sample(nodes, num_failures)
        
        # Create graph with these failures
        G_example = add_failure_status_to_graph(G, failed_nodes)
        
        # Add to examples
        examples.append(G_example)
    
    return examples

In [17]:
def predict_isolations(model, G):
    """Predict which nodes are isolated in a graph"""
    model.eval()
    device = next(model.parameters()).device
    
    # Prepare data
    data = prepare_data_from_graph(G)
    data = data.to(device)
    
    # Get node mapping
    node_list = list(G.nodes())
    
    # Make prediction
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        isolation_pred = (out.squeeze() > 0.5).cpu().numpy()
    
    # Get isolated nodes
    isolated_nodes = [node_list[i] for i, is_isolated in enumerate(isolation_pred) 
                     if is_isolated]
    
    return isolated_nodes

In [18]:
print("Generating training data...")
graph_examples = generate_graph_examples(G, num_examples=200)

# Train the model
print("Training model...")
model = train_graph_state_model(graph_examples)

# Create a test graph with specific failures
test_failures = ["NODE_0_0_2", "NODE_0_0_7"]
test_graph = add_failure_status_to_graph(G, test_failures)

# Predict isolations
print("\nPredicting isolations...")
isolated = predict_isolations(model, test_graph)

print(f"Failures: {test_failures}")
print(f"Isolated nodes: {len(isolated)}")
for node in isolated[:5]:
    print(f"- {node}")
if len(isolated) > 5:
    print(f"...and {len(isolated) - 5} more")

Generating training data...
Training model...
Epoch 1/30:
  Loss: 0.5981
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 2/30:
  Loss: 0.4847
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 3/30:
  Loss: 0.3464
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 4/30:
  Loss: 0.1888
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 5/30:
  Loss: 0.0772
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 6/30:
  Loss: 0.0317
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 7/30:
  Loss: 0.0212
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 8/30:
  Loss: 0.0190
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 9/30:
  Loss: 0.0193
  Precision: 0.0000, Recall: 0.0000, F1: 0.0000
  Predictions: 0.0/62000
Epoch 10/30:
  Loss: 0.0197
  Precision: 0.0000, Recal

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import networkx as nx
import pandas as pd
import random
import numpy as np

# Assuming your functions 'create_dummy_topology_data' and 'build_graph_with_position_features' are defined

# Create dummy data and graph
topology_df = create_dummy_topology_data(num_rings=10, nodes_per_ring=10, logical_rings_per_physical=3)
G_nx = build_graph_with_position_features(topology_df)

# Convert networkx graph to PyG Data object
def nx_to_pyg_data(G):
    # Map node features to vectors; here, we simply use a small feature vector based on our features
    mapping = {node: i for i, node in enumerate(G.nodes())}
    edge_index = []
    features = []
    for node, data in G.nodes(data=True):
        # Feature vector: [is_block, pr_id, lr_id, position]
        is_block = 1.0 if data.get('is_block', False) else 0.0
        pr_id = float(data.get('pr_id', -1))
        lr_id = float(data.get('lr_id', -1))
        position = float(data.get('position', -1))
        features.append([is_block, pr_id, lr_id, position])
    for u, v in G.edges():
        edge_index.append([mapping[u], mapping[v]])
        edge_index.append([mapping[v], mapping[u]])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.tensor(features, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)
    # Save mapping to refer back to node names if needed
    data.node_names = list(G.nodes())
    return data

data = nx_to_pyg_data(G_nx)

# Helper function: get ring membership based on node names and their features
def get_ring_id(node_name):
    # Assuming node_name format: "NODE_{pr}_{lr}_{pos}"
    parts = node_name.split('_')
    if len(parts) >= 4:
        return (int(parts[1]), int(parts[2]))
    return None

# Build a training dataset:
def create_training_sample(data, endpoints_per_sample=1):
    """
    For each training sample, we randomly select a ring, pick two endpoint nodes from that ring,
    and label nodes between them as class 1, others as class 0.
    """
    # Group nodes by ring using data.node_names and node feature information.
    ring_dict = {}
    for idx, node_name in enumerate(data.node_names):
        if "NODE_" in node_name:
            ring = get_ring_id(node_name)
            if ring is not None:
                ring_dict.setdefault(ring, []).append((idx, node_name))
    
    # Filter out rings with less than 2 nodes.
    rings = [nodes for nodes in ring_dict.values() if len(nodes) >= 2]
    if not rings:
        raise ValueError("Not enough nodes to form training samples.")
    
    # Select a random ring
    selected_ring = random.choice(rings)
    # Sort nodes in the ring by their position (extracted from the node name)
    selected_ring.sort(key=lambda x: int(x[1].split('_')[3]))
    # Randomly pick two endpoints ensuring the first is before the second
    i, j = sorted(random.sample(range(len(selected_ring)), 2))
    idx_a, name_a = selected_ring[i]
    idx_b, name_b = selected_ring[j]
    
    # Create labels for all nodes: label 1 if the node is in-between on the ring, else 0.
    labels = torch.zeros(data.num_nodes, dtype=torch.long)
    # Label the nodes in the selected ring between endpoints (including endpoints if desired)
    for idx, name in selected_ring[i:j+1]:
        labels[idx] = 1
    # Additionally, you can include the endpoints as context features if needed.
    return labels, (idx_a, idx_b)

# Create a simple dataset of training samples
class TopologyDataset(torch.utils.data.Dataset):
    def __init__(self, data, num_samples=1000):
        self.data = data
        self.num_samples = num_samples
        
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        # For each sample, generate new labels and return the full graph with sample-specific labels
        labels, endpoints = create_training_sample(self.data)
        # endpoints can be used in a more complex model as additional input; here we just return them
        sample = {
            'data': self.data, 
            'labels': labels,
            'endpoints': endpoints
        }
        return sample

dataset = TopologyDataset(data, num_samples=500)
loader = DataLoader(dataset, batch_size=1)

# Define a simple GNN Model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        # Node-level predictions
        out = self.lin(x)
        return out

# Instantiate model, loss and optimizer
model = GNNModel(in_channels=4, hidden_channels=16, out_channels=2)  # binary classification: in-between or not
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(model, loader, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for sample in loader:
            data_sample = sample['data'][0]  # since batch_size=1
            labels = sample['labels'][0]
            optimizer.zero_grad()
            out = model(data_sample)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Loss = {total_loss/len(loader):.4f}")

train(model, loader)



/home/nms/.local/lib/python3.13/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1: Loss = 0.0822
Epoch 2: Loss = 0.0772
Epoch 3: Loss = 0.0788
Epoch 4: Loss = 0.0785
Epoch 5: Loss = 0.0751
Epoch 6: Loss = 0.0760
Epoch 7: Loss = 0.0786
Epoch 8: Loss = 0.0770
Epoch 9: Loss = 0.0777
Epoch 10: Loss = 0.0800
Epoch 11: Loss = 0.0755
Epoch 12: Loss = 0.0780
Epoch 13: Loss = 0.0758
Epoch 14: Loss = 0.0782
Epoch 15: Loss = 0.0767
Epoch 16: Loss = 0.0769
Epoch 17: Loss = 0.0765
Epoch 18: Loss = 0.0790
Epoch 19: Loss = 0.0782
Epoch 20: Loss = 0.0753
Epoch 21: Loss = 0.0773
Epoch 22: Loss = 0.0783
Epoch 23: Loss = 0.0778
Epoch 24: Loss = 0.0782
Epoch 25: Loss = 0.0784
Epoch 26: Loss = 0.0778
Epoch 27: Loss = 0.0797
Epoch 28: Loss = 0.0777
Epoch 29: Loss = 0.0782
Epoch 30: Loss = 0.0773
Epoch 31: Loss = 0.0763
Epoch 32: Loss = 0.0786
Epoch 33: Loss = 0.0789
Epoch 34: Loss = 0.0771
Epoch 35: Loss = 0.0763
Epoch 36: Loss = 0.0772
Epoch 37: Loss = 0.0794
Epoch 38: Loss = 0.0790
Epoch 39: Loss = 0.0785
Epoch 40: Loss = 0.0769
Epoch 41: Loss = 0.0771
Epoch 42: Loss = 0.0749
E

In [20]:
def create_training_sample_with_marked_endpoints(data, endpoints_per_sample=1):
    """
    Create a training sample with explicitly marked endpoint nodes
    """
    # Group nodes by ring using data.node_names and node feature information
    ring_dict = {}
    for idx, node_name in enumerate(data.node_names):
        if "NODE_" in node_name:
            ring = get_ring_id(node_name)
            if ring is not None:
                ring_dict.setdefault(ring, []).append((idx, node_name))
    
    # Filter out rings with less than 2 nodes
    rings = [nodes for nodes in ring_dict.values() if len(nodes) >= 2]
    if not rings:
        raise ValueError("Not enough nodes to form training samples.")
    
    # Select a random ring
    selected_ring = random.choice(rings)
    
    # Sort nodes in the ring by their position
    selected_ring.sort(key=lambda x: int(x[1].split('_')[3]))
    
    # Randomly pick two endpoints ensuring the first is before the second
    i, j = sorted(random.sample(range(len(selected_ring)), 2))
    idx_a, name_a = selected_ring[i]
    idx_b, name_b = selected_ring[j]
    
    # Create endpoint markers for all nodes (initialized to zeros)
    endpoint_features = torch.zeros((data.num_nodes, 2), dtype=torch.float)
    
    # Mark endpoint nodes - first endpoint gets [1,0], second gets [0,1]
    endpoint_features[idx_a, 0] = 1.0
    endpoint_features[idx_b, 1] = 1.0
    
    # Create labels for nodes (1 if between endpoints, 0 otherwise)
    labels = torch.zeros(data.num_nodes, dtype=torch.long)
    
    # Label nodes between endpoints (not including endpoints themselves)
    for idx, name in selected_ring[i+1:j]:
        labels[idx] = 1
    
    # Create a copy of the data with added endpoint features
    new_x = torch.cat([data.x, endpoint_features], dim=1)
    new_data = Data(x=new_x, edge_index=data.edge_index)
    new_data.node_names = data.node_names
    
    return new_data, labels, (idx_a, idx_b)

In [21]:
class EnhancedTopologyDataset(torch.utils.data.Dataset):
    def __init__(self, base_data, num_samples=1000):
        self.base_data = base_data
        self.num_samples = num_samples
        
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        # For each sample, generate new data with marked endpoints and labels
        data_with_endpoints, labels, endpoints = create_training_sample_with_marked_endpoints(self.base_data)
        
        sample = {
            'data': data_with_endpoints, 
            'labels': labels,
            'endpoints': endpoints
        }
        return sample

In [22]:
class EndpointAwareGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(EndpointAwareGNN, self).__init__()
        # in_channels now includes the 2 additional endpoint marker features
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.attention = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )
        self.lin = nn.Linear(hidden_channels, out_channels)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First GCN layer
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        
        # Second GCN layer
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        
        # Node-level predictions
        out = self.lin(x)
        return out

In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def train_endpoint_aware(model, loader, epochs=50):
    model.train()
    
    # Use Adam optimizer with weight decay
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    for epoch in range(epochs):
        total_loss = 0
        all_preds = []
        all_labels = []
        
        for sample in loader:
            # Get data with endpoints already marked in features
            data_sample = sample['data'][0]  # since batch_size=1
            labels = sample['labels'][0]
            endpoints = sample['endpoints'][0]
            
            # Forward pass
            optimizer.zero_grad()
            out = model(data_sample)
            
            # Get predictions
            _, pred = out.max(dim=1)
            
            # Store predictions and labels for metrics calculation
            all_preds.append(pred.cpu())
            all_labels.append(labels.cpu())
            
            # Compute loss
            loss = criterion(out, labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Concatenate all predictions and labels
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        
        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, zero_division=0)
        recall = recall_score(all_labels, all_preds, zero_division=0)
        f1 = f1_score(all_labels, all_preds, zero_division=0)
        
        # Print epoch statistics
        print(f"Epoch {epoch+1}: Loss = {total_loss/len(loader):.4f}")
        print(f"  Accuracy = {accuracy:.4f}, Precision = {precision:.4f}")
        print(f"  Recall = {recall:.4f}, F1 Score = {f1:.4f}")
        # Save the model weights
        model_save_path = 'trained_gnn_model.pt'
        torch.save(model.state_dict(), model_save_path)
        print(f"  Model saved to {model_save_path}")




In [31]:
# Create the base data object
data = nx_to_pyg_data(G_nx)

# Create the enhanced dataset
dataset = EnhancedTopologyDataset(data, num_samples=500)
loader = DataLoader(dataset, batch_size=1)

# Create model with additional endpoint features
model = EndpointAwareGNN(
    in_channels=data.x.size(1) + 2,  # Original features + 2 endpoint markers
    hidden_channels=32, 
    out_channels=2
)

# Loss function
criterion = nn.CrossEntropyLoss()

# Train the model
train_endpoint_aware(model, loader, epochs=50)

/home/nms/.local/lib/python3.13/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1: Loss = 0.0543
  Accuracy = 0.9895, Precision = 0.0199
  Recall = 0.0036, F1 Score = 0.0061
  Model saved to trained_gnn_model.pt
Epoch 2: Loss = 0.0348
  Accuracy = 0.9912, Precision = 0.0000
  Recall = 0.0000, F1 Score = 0.0000
  Model saved to trained_gnn_model.pt
Epoch 3: Loss = 0.0326
  Accuracy = 0.9911, Precision = 0.0000
  Recall = 0.0000, F1 Score = 0.0000
  Model saved to trained_gnn_model.pt
Epoch 4: Loss = 0.0326
  Accuracy = 0.9912, Precision = 0.0000
  Recall = 0.0000, F1 Score = 0.0000
  Model saved to trained_gnn_model.pt
Epoch 5: Loss = 0.0291
  Accuracy = 0.9919, Precision = 0.0000
  Recall = 0.0000, F1 Score = 0.0000
  Model saved to trained_gnn_model.pt
Epoch 6: Loss = 0.0322
  Accuracy = 0.9909, Precision = 0.0000
  Recall = 0.0000, F1 Score = 0.0000
  Model saved to trained_gnn_model.pt
Epoch 7: Loss = 0.0302
  Accuracy = 0.9912, Precision = 0.0000
  Recall = 0.0000, F1 Score = 0.0000
  Model saved to trained_gnn_model.pt
Epoch 8: Loss = 0.0286
  Accuracy 

KeyboardInterrupt: 

In [28]:
def infer_for_specific_nodes(model, data, node_indices):
    """
    Get predictions for specific nodes from an already prepared graph
    
    Args:
        model: Trained GNNModel
        data: PyG Data object containing the graph
        node_indices: Indices of the nodes to get predictions for
        
    Returns:
        Tensor of predictions for specified nodes
    """
    model.eval()
    
    with torch.no_grad():
        # Forward pass on the entire graph
        all_node_predictions = model(data)
        
        # Extract only the predictions for specified nodes
        node_predictions = all_node_predictions[node_indices]
    
    return node_predictions

In [32]:
# Load your trained model
model = GNNModel(in_channels=5, hidden_channels=64, out_channels=1)
model.load_state_dict(torch.load('trained_gnn_model.pt'))

# Define input nodes
node1 = "NODE_0_0_2"
node2 = "NODE_0_0_7"

# Make predictions
results = predict_for_two_nodes(model, G, node1, node2)

# Print results
print(f"Input nodes: {results['input_nodes']}")
print(f"Affected nodes: {len(results['affected_nodes'])}")
for node, prob in results['affected_nodes'][:5]:
    print(f"- {node}: {prob:.4f}")

RuntimeError: Error(s) in loading state_dict for GNNModel:
	Unexpected key(s) in state_dict: "attention.0.weight", "attention.0.bias", "attention.2.weight", "attention.2.bias". 
	size mismatch for conv1.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for conv1.lin.weight: copying a param with shape torch.Size([32, 6]) from checkpoint, the shape in current model is torch.Size([64, 5]).
	size mismatch for conv2.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for conv2.lin.weight: copying a param with shape torch.Size([32, 32]) from checkpoint, the shape in current model is torch.Size([64, 64]).
	size mismatch for lin.weight: copying a param with shape torch.Size([2, 32]) from checkpoint, the shape in current model is torch.Size([1, 64]).
	size mismatch for lin.bias: copying a param with shape torch.Size([2]) from checkpoint, t

In [15]:
from torch_geometric.loader import DataLoader  # Use PyG's DataLoader

class SingleGraphDataset(torch.utils.data.Dataset):
    def __init__(self, num_samples=1000, num_rings=10, nodes_per_ring=10, logical_rings_per_physical=3):
        # Create graph
        topology_df = create_dummy_topology_data(num_rings, nodes_per_ring, logical_rings_per_physical)
        self.G = build_graph_with_position_features(topology_df)
        self.num_samples = num_samples
        
        # Precompute static graph data
        self.node_list = list(self.G.nodes())
        
        # Create node name to index mapping
        self.node_to_idx = {node: i for i, node in enumerate(self.node_list)}
        
        # Convert edges to index pairs
        edge_list = []
        for u, v in self.G.edges():
            edge_list.append([self.node_to_idx[u], self.node_to_idx[v]])
            edge_list.append([self.node_to_idx[v], self.node_to_idx[u]])
        
        self.edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
        
        # Create node features
        node_features = []
        for node in self.node_list:
            attrs = self.G.nodes[node]
            features = [
                float(attrs.get('is_block', False)),
                attrs.get('pr_id', -1) / 10.0,
                attrs.get('lr_id', -1) / 10.0,
                attrs.get('position', -1) / 10.0,
            ]
            node_features.append(features)
        
        self.node_features = torch.tensor(node_features, dtype=torch.float)
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        # Generate random sample using the static graph
        from torch_geometric.data import Data
        
        # Select random ring and two nodes
        rings = {}
        for node, data in self.G.nodes(data=True):
            pr_id = data.get('pr_id', -1)
            lr_id = data.get('lr_id', -1)
            if pr_id >= 0 and lr_id >= 0:
                key = (pr_id, lr_id)
                if key not in rings:
                    rings[key] = []
                rings[key].append((node, data.get('position', -1)))
        
        # Filter rings with enough nodes
        valid_rings = [nodes for nodes in rings.values() if len(nodes) >= 2]
        if not valid_rings:
            # Fallback if no valid rings
            return Data(x=self.node_features, edge_index=self.edge_index, 
                       y=torch.zeros(len(self.node_list), dtype=torch.long))
        
        # Select a random ring
        selected_ring = random.choice(valid_rings)
        
        # Sort by position
        selected_ring.sort(key=lambda x: x[1])
        
        # Pick two random nodes as endpoints
        if len(selected_ring) < 2:
            endpoint_indices = [0, 0]  # Fallback
        else:
            i, j = sorted(random.sample(range(len(selected_ring)), 2))
            endpoint_indices = [self.node_to_idx[selected_ring[i][0]], 
                              self.node_to_idx[selected_ring[j][0]]]
        
        # Create endpoint markers
        endpoint_markers = torch.zeros((len(self.node_list), 2), dtype=torch.float)
        endpoint_markers[endpoint_indices[0], 0] = 1.0
        endpoint_markers[endpoint_indices[1], 1] = 1.0
        
        # Combine features with endpoint markers
        x = torch.cat([self.node_features, endpoint_markers], dim=1)
        
        # Create labels - nodes between endpoints are isolated
        y = torch.zeros(len(self.node_list), dtype=torch.long)
        
        # Only label if we have two distinct endpoints
        if endpoint_indices[0] != endpoint_indices[1]:
            # Get all nodes in the ring
            ring_nodes = [self.node_to_idx[node] for node, _ in selected_ring]
            
            # Get min/max index in the sorted ring
            min_idx = min(endpoint_indices)
            max_idx = max(endpoint_indices)
            
            # Label nodes between endpoints in the ring
            ring_pos_dict = {self.node_to_idx[node]: pos for node, pos in selected_ring}
            min_pos = ring_pos_dict[min_idx]
            max_pos = ring_pos_dict[max_idx]
            
            for node_idx in ring_nodes:
                if node_idx in endpoint_indices:
                    continue
                pos = ring_pos_dict.get(node_idx, -1)
                if min_pos < pos < max_pos:
                    y[node_idx] = 1
        
        return Data(x=x, edge_index=self.edge_index, y=y, endpoints=torch.tensor(endpoint_indices))

In [16]:

# The GNN model remains the same as previous
class GNNModel(nn.Module):
    def __init__(self, feature_dim=6, hidden_dim=64, output_dim=1):
        super().__init__()
        self.conv1 = GATConv(feature_dim, hidden_dim)
        self.conv2 = GATConv(hidden_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = x.float()
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.classifier(x)
        return torch.sigmoid(x.squeeze())

def train_single_graph():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Create dataset
    dataset = SingleGraphDataset(num_samples=1000)
    
    # Use PyG's DataLoader
    from torch_geometric.loader import DataLoader
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Define model
    model = GNNModel(in_channels=dataset.node_features.size(1) + 2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(50):
        model.train()
        total_loss = 0
        all_preds = []
        all_labels = []
        
        for batch in loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            out = model(batch)
            
            # Get predictions
            _, pred = out.max(dim=1)
            
            # Store for metrics
            all_preds.append(pred.cpu())
            all_labels.append(batch.y.cpu())
            
            # Loss and backward
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Calculate metrics
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        
        from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, zero_division=0)
        recall = recall_score(all_labels, all_preds, zero_division=0)
        f1 = f1_score(all_labels, all_preds, zero_division=0)
        
        print(f"Epoch {epoch+1}:")
        print(f"  Loss = {total_loss/len(loader):.4f}")
        print(f"  Accuracy = {accuracy:.4f}, Precision = {precision:.4f}")
        print(f"  Recall = {recall:.4f}, F1 Score = {f1:.4f}")
        print(f"  Positive predictions: {all_preds.sum().item()}/{len(all_preds)}")

if __name__ == '__main__':
    train_single_graph()

TypeError: GNNModel.__init__() got an unexpected keyword argument 'in_channels'

In [53]:
import torch
from torch_geometric.data import Dataset, Data
import random

class NodeIsolationDataset(Dataset):
    def __init__(self,G_nx, num_samples=1000, transform=None, pre_transform=None):
        super(NodeIsolationDataset, self).__init__(None, transform, pre_transform)
        
        # Create base graph
        
        self.G = G_nx
        self.num_samples = num_samples
        
        # Create mapping from node names to indices
        self.node_list = list(self.G.nodes())
        self.node_to_idx = {node: i for i, node in enumerate(self.node_list)}
        
        # Extract ring information for easier access
        self.rings = {}
        for node, data in self.G.nodes(data=True):
            pr_id = data.get('pr_id')
            lr_id = data.get('lr_id')
            position = data.get('position')
            
            if (pr_id, lr_id) not in self.rings:
                self.rings[(pr_id, lr_id)] = []
            
            self.rings[(pr_id, lr_id)].append((node, position))
        
        # Pre-build edge index
        edge_index = []
        for src, dst in self.G.edges():
            src_idx = self.node_to_idx[src]
            dst_idx = self.node_to_idx[dst]
            edge_index.append([src_idx, dst_idx])
            edge_index.append([dst_idx, src_idx])  # Undirected graph
        
        self.edge_index = torch.tensor(edge_index, dtype=torch.long).t()
        
        # Pre-build node features (without failure markers)
        base_features = []
        for node in self.node_list:
            data = self.G.nodes[node]
            features = [
                float(data.get('is_block', False)),
                data.get('pr_id', -1),  # Normalize
                data.get('lr_id', -1) ,  # Normalize
                data.get('position', -1),  # Normalize
            ]
            base_features.append(features)
        
        self.base_features = torch.tensor(base_features, dtype=torch.float)
    
    def len(self):
        return self.num_samples
    
    def get(self, idx):
        # Generate a new sample for each index
        # 1. Select a random ring
        valid_rings = [(pr_lr, nodes) for pr_lr, nodes in self.rings.items() if len(nodes) >= 3]
        
        if not valid_rings:
            # Fallback if no valid rings
            return Data(
                x=self.base_features, 
                edge_index=self.edge_index,
                y=torch.zeros(len(self.node_list), dtype=torch.long)
            )
        
        ring_key, ring_nodes = random.choice(valid_rings)
        
        # 2. Pick two random nodes from the ring to mark as failed
        sorted_ring_nodes = sorted(ring_nodes, key=lambda x: x[1])  # Sort by position
        
        if len(sorted_ring_nodes) < 2:
            # Safety check
            i, j = 0, 0
        else:
            i, j = sorted(random.sample(range(len(sorted_ring_nodes)), 2))
        
        # Get the failed nodes
        failed_node1, pos1 = sorted_ring_nodes[i]
        failed_node2, pos2 = sorted_ring_nodes[j]
        
        # Get indices in our node list
        failed_idx1 = self.node_to_idx[failed_node1]
        failed_idx2 = self.node_to_idx[failed_node2]
        
        # 3. Create failure marker features (2 additional features)
        failure_markers = torch.zeros((len(self.node_list), 2), dtype=torch.float)
        failure_markers[failed_idx1, 0] = 1.0  # First failed node
        failure_markers[failed_idx2, 1] = 1.0  # Second failed node
        
        # 4. Combine features
        x = torch.cat([self.base_features, failure_markers], dim=1)
        
        # 5. Create label tensor - nodes between the two failed nodes should be isolated
        y = torch.zeros(len(self.node_list), dtype=torch.long)
        
        # Only if nodes are in the same ring
        min_pos = min(pos1, pos2)
        max_pos = max(pos1, pos2)
        
        # Find nodes that should be isolated (in same ring with position between failed nodes)
        for node, pos in sorted_ring_nodes:
            if node in [failed_node1, failed_node2]:
                continue  # Skip failed nodes
            
            # Check if position is between the failed nodes
            if min_pos < pos < max_pos:
                # Mark as isolated (target = 1)
                node_idx = self.node_to_idx[node]
                y[node_idx] = 1
        
        # 6. Create and return Data object
        return Data(
            x=x,
            edge_index=self.edge_index,
            y=y,
            failed_nodes=torch.tensor([failed_idx1, failed_idx2], dtype=torch.long)
        )

In [54]:
class IsolationGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels=64):
        super(IsolationGNN, self).__init__()
        
        # GNN layers
        self.conv1 = GCNConv(in_channels, hidden_channels)
        
        # FIXED: Specify the correct output dimensions and heads
        self.conv2 = GATConv(hidden_channels, hidden_channels, heads=2)
        
        # FIXED: Match the output dimensions from GAT (hidden_channels * heads)
        self.classifier = nn.Linear(hidden_channels * 2, 2)  # Binary classification
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First layer with ReLU activation
        h = torch.relu(self.conv1(x, edge_index))
        
        # Second layer
        h = self.conv2(h, edge_index)  # Output shape: [nodes, heads, hidden_channels]
        
        # FIXED: Handle output shape from GAT correctly
        # If h shape is [nodes, heads, features], reshape to [nodes, heads*features]
        if len(h.shape) == 3:
            batch_size, heads, features = h.shape
            h = h.reshape(batch_size, heads * features)
        
        # Final classification
        out = self.classifier(h)
        
        return out

In [41]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [55]:
def train_isolation_model(G_nx,num_epochs=30, batch_size=32):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Create dataset
    dataset = NodeIsolationDataset(G_nx,num_samples=1000)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Create model
    model = IsolationGNN(in_channels=dataset.base_features.size(1) + 2).to(device)
    
    # Optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 5.0]).to(device))  # Weight positive class higher
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        all_preds = []
        all_labels = []
        
        for batch in loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            out = model(batch)
            
            # Compute loss
            loss = criterion(out, batch.y)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Get predictions
            _, pred = out.max(dim=1)
            
            # Track predictions and labels for metrics
            all_preds.append(pred.cpu())
            all_labels.append(batch.y.cpu())
            
            total_loss += loss.item()
        
        # Combine predictions and labels
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        
        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, zero_division=0)
        recall = recall_score(all_labels, all_preds, zero_division=0)
        f1 = f1_score(all_labels, all_preds, zero_division=0)
        
        # Print metrics
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Loss: {total_loss/len(loader):.4f}")
        print(f"  Accuracy: {accuracy:.4f}, Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}, F1 Score: {f1:.4f}")
        print(f"  Positive predictions: {all_preds.sum().item()}/{len(all_preds)}")
    
    return model

In [ ]:
def predict_isolated_nodes(model, graph, failed_node1, failed_node2):
    """
    Predict which nodes should be isolated when two nodes fail
    
    Args:
        model: Trained IsolationGNN model
        graph: NetworkX graph with node attributes
        failed_node1, failed_node2: Names of the two failed nodes
        
    Returns:
        List of node names predicted to be isolated
    """
    model.eval()
    
    # Convert graph to appropriate format
    node_list = list(graph.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Check if failed nodes exist in the graph
    if failed_node1 not in node_to_idx or failed_node2 not in node_to_idx:
        print(f"Error: One or both failed nodes not in graph")
        return []
    
    # Get indices of failed nodes
    failed_idx1 = node_to_idx[failed_node1]
    failed_idx2 = node_to_idx[failed_node2]
    
    # Create edge index
    edge_index = []
    for u, v in graph.edges():
        edge_index.append([node_to_idx[u], node_to_idx[v]])
        edge_index.append([node_to_idx[v], node_to_idx[u]])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t()
    
    # Create node features
    base_features = []
    for node in node_list:
        data = graph.nodes[node]
        features = [
            float(data.get('is_block', False)),
            data.get('pr_id', -1) ,
            data.get('lr_id', -1),
            data.get('position', -1)
        ]
        base_features.append(features)
    
    base_features = torch.tensor(base_features, dtype=torch.float)
    
    # Create failure markers
    failure_markers = torch.zeros((len(node_list), 2), dtype=torch.float)
    failure_markers[failed_idx1, 0] = 1.0
    failure_markers[failed_idx2, 1] = 1.0
    
    # Combine features
    x = torch.cat([base_features, failure_markers], dim=1)
    
    # Create data object
    data = Data(x=x, edge_index=edge_index)
    
    # Make prediction
    with torch.no_grad():
        out = model(data)
        for i, node_name in enumerate(node_list):  # first 10 nodes
            print(f"{node_name}: {out[i].tolist()}")
        _, pred = out.max(dim=1)
    
    # Get isolated nodes
    isolated_indices = torch.nonzero(pred == 1).squeeze().tolist()
    
    # Convert to list if single item
    if not isinstance(isolated_indices, list):
        isolated_indices = [isolated_indices]
    
    # Convert to node names and exclude failed nodes
    isolated_nodes = [node_list[idx] for idx in isolated_indices 
                    if idx != failed_idx1 and idx != failed_idx2]
    
    return isolated_nodes

In [30]:
topo_data = create_dummy_topology_data()

In [31]:
G_nx = build_graph_with_position_features(topo_data)

In [56]:
model = train_isolation_model(G_nx,num_epochs=30, batch_size=32)

/home/nms/.local/lib/python3.13/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1/30:
  Loss: 0.2472
  Accuracy: 0.9827, Precision: 0.0112
  Recall: 0.0109, F1 Score: 0.0110
  Positive predictions: 2687/310000
Epoch 2/30:
  Loss: 0.1916
  Accuracy: 0.9914, Precision: 0.0000
  Recall: 0.0000, F1 Score: 0.0000
  Positive predictions: 0/310000
Epoch 3/30:
  Loss: 0.1767
  Accuracy: 0.9918, Precision: 0.0000
  Recall: 0.0000, F1 Score: 0.0000
  Positive predictions: 0/310000
Epoch 4/30:
  Loss: 0.1706
  Accuracy: 0.9919, Precision: 0.0000
  Recall: 0.0000, F1 Score: 0.0000
  Positive predictions: 0/310000
Epoch 5/30:
  Loss: 0.1675
  Accuracy: 0.9919, Precision: 0.0000
  Recall: 0.0000, F1 Score: 0.0000
  Positive predictions: 0/310000
Epoch 6/30:
  Loss: 0.1652
  Accuracy: 0.9915, Precision: 0.0000
  Recall: 0.0000, F1 Score: 0.0000
  Positive predictions: 0/310000
Epoch 7/30:
  Loss: 0.1580
  Accuracy: 0.9917, Precision: 0.0000
  Recall: 0.0000, F1 Score: 0.0000
  Positive predictions: 0/310000
Epoch 8/30:
  Loss: 0.1424
  Accuracy: 0.9918, Precision: 0.0000
 

In [58]:
failed_node1 = "NODE_1_0_4"
failed_node2 = "NODE_1_0_6"
    
    # Predict isolated nodes
isolated = predict_isolated_nodes(model, G_nx, failed_node1, failed_node2)
print(f"\nNodes isolated by {failed_node1} and {failed_node2}:")
for node in isolated:
        print(f"- {node}")


NODE_0_0_0: [2.3151636123657227, -2.1297192573547363]
NODE_0_0_1: [2.1137428283691406, -1.9471200704574585]
NODE_0_0_2: [1.825444221496582, -1.6798732280731201]
NODE_0_0_3: [1.8049228191375732, -1.6564351320266724]
NODE_0_0_4: [1.7815285921096802, -1.6319700479507446]
NODE_0_0_5: [1.755421757698059, -1.606775164604187]
NODE_0_0_6: [1.7244935035705566, -1.5778577327728271]
NODE_0_0_7: [1.6938259601593018, -1.5495681762695312]
NODE_0_0_8: [1.9507694244384766, -1.7858049869537354]
NODE_0_0_9: [2.1780738830566406, -1.9944514036178589]
BLOCK_0: [2.6768946647644043, -2.457437753677368]
NODE_0_1_0: [2.3206024169921875, -2.137464761734009]
NODE_0_1_1: [2.1223325729370117, -1.9592417478561401]
NODE_0_1_2: [1.8352240324020386, -1.6932729482650757]
NODE_0_1_3: [1.8099979162216187, -1.6647611856460571]
NODE_0_1_4: [1.78563392162323, -1.638372778892517]
NODE_0_1_5: [1.7629214525222778, -1.6152315139770508]
NODE_0_1_6: [1.7371735572814941, -1.590527892112732]
NODE_0_1_7: [1.7101103067398071, -1.5653

In [59]:
class PathBasedIsolationGNN(nn.Module):
    """GNN that learns to identify nodes on paths between two input nodes"""
    def __init__(self, in_channels, hidden_channels=64):
        super(PathBasedIsolationGNN, self).__init__()
        
        # Node embedding
        self.node_encoder = nn.Linear(in_channels, hidden_channels)
        
        # Message passing layers
        self.conv1 = GCNConv(hidden_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        
        # Path awareness layer
        self.path_layer = nn.Linear(hidden_channels, hidden_channels)
        
        # Output classification
        self.classifier = nn.Linear(hidden_channels, 2)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Initial node features encoding
        h = self.node_encoder(x)
        
        # Message passing to understand graph structure
        h1 = F.relu(self.conv1(h, edge_index))
        h2 = F.relu(self.conv2(h1, edge_index)) + h1  # Residual connection
        h3 = F.relu(self.conv3(h2, edge_index)) + h2  # Residual connection
        
        # Path awareness
        h_path = F.relu(self.path_layer(h3))
        
        # Final classification
        out = self.classifier(h_path)
        
        return out

In [60]:
def create_path_based_training_data(G, num_samples=1000):
    """Create training data focused on graph paths"""
    from torch_geometric.data import Data
    from torch_geometric.utils import to_networkx
    
    data_list = []
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Create edge index
    edge_index = []
    for u, v in G.edges():
        edge_index.append([node_to_idx[u], node_to_idx[v]])
        edge_index.append([node_to_idx[v], node_to_idx[u]])  # Undirected graph
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t()
    
    # Extract ring information
    rings = {}
    for node, data in G.nodes(data=True):
        pr_id = data.get('pr_id')
        lr_id = data.get('lr_id')
        position = data.get('position')
        
        if pr_id is not None and lr_id is not None and position is not None:
            if (pr_id, lr_id) not in rings:
                rings[(pr_id, lr_id)] = []
            
            rings[(pr_id, lr_id)].append((node, position))
    
    # Generate samples
    for _ in range(num_samples):
        # Create base features
        base_features = []
        for node in node_list:
            data = G.nodes[node]
            features = [
                float(data.get('is_block', False)),
                data.get('pr_id', -1) / 10.0,
                data.get('lr_id', -1) / 10.0,
                data.get('position', -1) / 10.0
            ]
            base_features.append(features)
        
        # Select random ring and two nodes
        valid_rings = [(k, v) for k, v in rings.items() if len(v) >= 3]
        if not valid_rings:
            continue
            
        ring_key, ring_nodes = random.choice(valid_rings)
        
        # Sort by position
        sorted_ring_nodes = sorted(ring_nodes, key=lambda x: x[1])
        
        # Choose two nodes as endpoints
        if len(sorted_ring_nodes) < 2:
            continue
            
        i, j = sorted(random.sample(range(len(sorted_ring_nodes)), 2))
        endpoint1, _ = sorted_ring_nodes[i]
        endpoint2, _ = sorted_ring_nodes[j]
        
        # Mark endpoints in features
        endpoint_markers = torch.zeros((len(node_list), 2), dtype=torch.float)
        endpoint_markers[node_to_idx[endpoint1], 0] = 1.0
        endpoint_markers[node_to_idx[endpoint2], 1] = 1.0
        
        # Combine features
        features = torch.tensor(base_features, dtype=torch.float)
        x = torch.cat([features, endpoint_markers], dim=1)
        
        # Find nodes that should be isolated (on path between endpoints in the ring)
        path_nodes = []
        for idx in range(i+1, j):
            node, _ = sorted_ring_nodes[idx]
            path_nodes.append(node_to_idx[node])
        
        # Create labels
        y = torch.zeros(len(node_list), dtype=torch.long)
        for idx in path_nodes:
            y[idx] = 1
        
        # Create data object with graph structure as primary information
        data = Data(x=x, edge_index=edge_index, y=y, 
                    endpoints=torch.tensor([node_to_idx[endpoint1], node_to_idx[endpoint2]]))
        
        data_list.append(data)
    
    return data_list

In [61]:
def train_path_isolation_model(G_nx, num_epochs=30, batch_size=32):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Create dataset focused on graph paths
    dataset = create_path_based_training_data(G_nx, num_samples=1000)
    
    # Split train/validation
    from sklearn.model_selection import train_test_split
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
    
    # Create data loaders
    from torch_geometric.loader import DataLoader
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)
    
    # Create model
    model = PathBasedIsolationGNN(in_channels=6).to(device)  # 4 node features + 2 endpoint markers
    
    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 10.0]).to(device))  # Weight positive class more
    
    best_f1 = 0.0
    best_model = None
    
    # Training loop
    for epoch in range(num_epochs):
        # Train
        model.train()
        total_loss = 0
        train_preds, train_labels = [], []
        
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            out = model(batch)
            
            # Get predictions
            _, pred = out.max(dim=1)
            
            # Store predictions and labels
            train_preds.append(pred.cpu())
            train_labels.append(batch.y.cpu())
            
            # Compute loss and backward
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
        # Validation
        model.eval()
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch)
                _, pred = out.max(dim=1)
                
                val_preds.append(pred.cpu())
                val_labels.append(batch.y.cpu())
        
        # Calculate metrics
        from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
        
        # Training metrics
        train_preds = torch.cat(train_preds)
        train_labels = torch.cat(train_labels)
        train_acc = accuracy_score(train_labels, train_preds)
        train_prec = precision_score(train_labels, train_preds, zero_division=0)
        train_rec = recall_score(train_labels, train_preds, zero_division=0)
        train_f1 = f1_score(train_labels, train_preds, zero_division=0)
        
        # Validation metrics
        val_preds = torch.cat(val_preds)
        val_labels = torch.cat(val_labels)
        val_acc = accuracy_score(val_labels, val_preds)
        val_prec = precision_score(val_labels, val_preds, zero_division=0)
        val_rec = recall_score(val_labels, val_preds, zero_division=0)
        val_f1 = f1_score(val_labels, val_preds, zero_division=0)
        
        # Save best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model = model.state_dict().copy()
        
        # Print metrics
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train: Loss={total_loss/len(train_loader):.4f}, F1={train_f1:.4f}")
        print(f"  Train: Precision={train_prec:.4f}, Recall={train_rec:.4f}")
        print(f"  Val: F1={val_f1:.4f}, Precision={val_prec:.4f}, Recall={val_rec:.4f}")
        print(f"  Positives: {val_preds.sum().item()}/{len(val_preds)}")
    
    # Load best model
    if best_model:
        model.load_state_dict(best_model)
    
    return model

In [82]:
def predict_path_isolation(model, G, node1, node2):
    """Predict nodes on the path between two specified nodes"""
    # Setup
    model.eval()
    device = next(model.parameters()).device
    
    # Convert graph to indices
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Check if nodes exist
    if node1 not in node_to_idx or node2 not in node_to_idx:
        print(f"Error: One or both endpoints not found in graph")
        return []
    
    # Create edge index
    edge_index = []
    for u, v in G.edges():
        edge_index.append([node_to_idx[u], node_to_idx[v]])
        edge_index.append([node_to_idx[v], node_to_idx[u]])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().to(device)
    
    # Create node features
    base_features = []
    for node in node_list:
        data = G.nodes[node]
        features = [
            float(data.get('is_block', False)),
            data.get('pr_id', -1) / 10.0,
            data.get('lr_id', -1) / 10.0,
            data.get('position', -1) / 10.0
        ]
        base_features.append(features)
    
    # Create endpoint markers
    endpoint_markers = torch.zeros((len(node_list), 2), dtype=torch.float)
    endpoint_markers[node_to_idx[node1], 0] = 1.0
    endpoint_markers[node_to_idx[node2], 1] = 1.0
    
    # Combine features
    features = torch.tensor(base_features, dtype=torch.float).to(device)
    x = torch.cat([features, endpoint_markers.to(device)], dim=1)
    
    # Create data object
    from torch_geometric.data import Data
    data = Data(x=x, edge_index=edge_index)
    
    # Make prediction
    with torch.no_grad():
        out = model(data)
        probs = torch.softmax(out, dim=1)
        _, pred = out.max(dim=1)
    
    # Get predicted isolated nodes
    isolated_indices = torch.nonzero(pred == 1).squeeze().cpu().tolist()
    
    # Handle case of single or no result
    if not isinstance(isolated_indices, list):
        isolated_indices = [isolated_indices] if isolated_indices.numel() > 0 else []
    
    # Convert to node names
    isolated_nodes = [node_list[idx] for idx in isolated_indices 
                    ]
    
    return isolated_nodes, out, node_list

In [93]:
model2 = train_path_isolation_model(G_nx, num_epochs=100, batch_size=32)

Epoch 1/100:
  Train: Loss=0.4303, F1=0.0208
  Train: Precision=0.0115, Recall=0.1088
  Val: F1=0.0000, Precision=0.0000, Recall=0.0000
  Positives: 0/62000
Epoch 2/100:
  Train: Loss=0.2829, F1=0.0000
  Train: Precision=0.0000, Recall=0.0000
  Val: F1=0.0000, Precision=0.0000, Recall=0.0000
  Positives: 0/62000
Epoch 3/100:
  Train: Loss=0.2660, F1=0.0000
  Train: Precision=0.0000, Recall=0.0000
  Val: F1=0.0000, Precision=0.0000, Recall=0.0000
  Positives: 0/62000
Epoch 4/100:
  Train: Loss=0.2445, F1=0.0000
  Train: Precision=0.0000, Recall=0.0000
  Val: F1=0.0000, Precision=0.0000, Recall=0.0000
  Positives: 0/62000
Epoch 5/100:
  Train: Loss=0.2031, F1=0.0072
  Train: Precision=0.2051, Recall=0.0037
  Val: F1=0.1397, Precision=0.3761, Recall=0.0858
  Positives: 117/62000
Epoch 6/100:
  Train: Loss=0.1487, F1=0.3605
  Train: Precision=0.2990, Recall=0.4539
  Val: F1=0.4321, Precision=0.3219, Recall=0.6569
  Positives: 1047/62000
Epoch 7/100:
  Train: Loss=0.1215, F1=0.4570
  Train:

In [97]:
node1 = "NODE_1_0_4"
node2 = "NODE_1_0_5"
isolated,out,node_list = predict_path_isolation(model2, G_nx, node1, node2)
isolated

[]

In [81]:
isolated

['NODE_5_0_3', 'NODE_5_0_5', 'NODE_5_0_7']

In [79]:
isolated

['NODE_3_0_3', 'NODE_3_0_5', 'NODE_3_0_7']

In [73]:
for i, node_name in enumerate(node_list):  # first 10 nodes
    print(f"{node_name}: {out[i].tolist()}")
    _, pred = out.max(dim=1)

NODE_0_0_0: [3.255542039871216, -2.4851021766662598]
NODE_0_0_1: [3.2551181316375732, -2.7131824493408203]
NODE_0_0_2: [4.006722450256348, -3.4877960681915283]
NODE_0_0_3: [2.879310369491577, -2.453397274017334]
NODE_0_0_4: [2.2853047847747803, -1.882545828819275]
NODE_0_0_5: [2.141958475112915, -1.7502539157867432]
NODE_0_0_6: [2.580329656600952, -2.165238857269287]
NODE_0_0_7: [4.0471510887146, -3.5342583656311035]
NODE_0_0_8: [3.4005801677703857, -2.8939390182495117]
NODE_0_0_9: [3.3036420345306396, -2.563901901245117]
BLOCK_0: [1.7650765180587769, -0.8555459976196289]
NODE_0_1_0: [3.281153440475464, -2.5047178268432617]
NODE_0_1_1: [3.3073740005493164, -2.7558741569519043]
NODE_0_1_2: [4.066165447235107, -3.5339479446411133]
NODE_0_1_3: [2.893030881881714, -2.454338550567627]
NODE_0_1_4: [2.3501172065734863, -1.93293035030365]
NODE_0_1_5: [2.2769405841827393, -1.866085410118103]
NODE_0_1_6: [2.7471938133239746, -2.3133931159973145]
NODE_0_1_7: [4.133645057678223, -3.603438854217529

In [140]:
class SimplePathGNN(nn.Module):
    """GNN with improved single-node path detection"""
    def __init__(self, hidden_channels=64):
        super(SimplePathGNN, self).__init__()
        
        # Input layers
        self.conv1 = GCNConv(2, hidden_channels)
        
        # Direct connection layer - specifically for single-hop connections
        self.direct_layer = GCNConv(hidden_channels, hidden_channels//2)
        
        # Multi-hop layers
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        
        # Output MLP
        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels + hidden_channels//2, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 2)
        )
    
    def forward(self, x, edge_index):
        # Initial embedding
        h = F.relu(self.conv1(x, edge_index))
        
        # Direct connection path - capture one-hop paths
        h_direct = F.relu(self.direct_layer(h, edge_index))
        
        # Multi-hop path
        h = F.relu(self.conv2(h, edge_index))
        h = F.relu(self.conv3(h, edge_index))
        
        # Combine both paths
        h_combined = torch.cat([h, h_direct], dim=1)
        
        # Final classification
        out = self.classifier(h_combined)
        
        return out

In [124]:
def create_path_samples(G, num_samples=1000):
    """Create samples where labels are nodes on paths between two input nodes"""
    import networkx as nx
    from torch_geometric.data import Data
    
    samples = []
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Create edge index
    edge_index = []
    for u, v in G.edges():
        edge_index.append([node_to_idx[u], node_to_idx[v]])
        edge_index.append([node_to_idx[v], node_to_idx[u]])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t()
    
    # Group nodes by ring (PR and LR)
    rings = {}
    for node in G.nodes():
        data = G.nodes[node]
        pr_id = data.get('pr_id')
        lr_id = data.get('lr_id')
        
        if pr_id is not None and lr_id is not None:
            key = (pr_id, lr_id)
            if key not in rings:
                rings[key] = []
            rings[key].append(node)
    
    # Filter rings with enough nodes
    valid_rings = {key: nodes for key, nodes in rings.items() if len(nodes) >= 2}
    
    # Create NetworkX graph for path finding
    G_nx = nx.Graph()
    for u, v in G.edges():
        G_nx.add_edge(u, v)
    
    # Generate samples
    sample_count = 0
    attempts = 0
    max_attempts = num_samples * 5
    
    while sample_count < num_samples and attempts < max_attempts:
        attempts += 1
        
        # CHANGED: Select a random ring and two nodes from it
        if not valid_rings:
            print("Warning: No valid rings found with at least 2 nodes")
            break
            
        ring_key = random.choice(list(valid_rings.keys()))
        ring_nodes = valid_rings[ring_key]
        
        # Choose two random nodes from this ring
        node1, node2 = random.sample(ring_nodes, 2)
        
        # Check if a path exists
        if not nx.has_path(G_nx, node1, node2):
            continue
        
        # Find shortest path
        path = nx.shortest_path(G_nx, node1, node2)
        
        # Create input features (just the two marker nodes)
        x = torch.zeros((len(node_list), 2), dtype=torch.float)
        x[node_to_idx[node1], 0] = 1.0  # Mark first input node
        x[node_to_idx[node2], 1] = 1.0  # Mark second input node
        
        # Create labels (nodes on the path)
        y = torch.zeros(len(node_list), dtype=torch.long)
        
        # Mark nodes on the path (excluding endpoints)
        for node in path[1:-1]:
            y[node_to_idx[node]] = 1
        
        # Create data object
        data = Data(
            x=x, 
            edge_index=edge_index, 
            y=y,
            path_nodes=[node_to_idx[n] for n in path]  # Store for reference
        )
        
        samples.append(data)
        sample_count += 1
    
    print(f"Generated {len(samples)} samples from {len(valid_rings)} unique rings")
    return samples

In [121]:
def train_simple_path_model(G, num_epochs=30, batch_size=32):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Create dataset
    print("Creating path samples...")
    dataset = create_path_samples(G, num_samples=2000)
    
    # Split train/val
    from sklearn.model_selection import train_test_split
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
    
    # Create loaders
    from torch_geometric.loader import DataLoader
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)
    
    # Create model
    model = SimplePathGNN(hidden_channels=64).to(device)
    
    # Optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 5.0]).to(device))
    
    # Training loop
    print("Training model...")
    for epoch in range(num_epochs):
        # Train
        model.train()
        total_loss = 0
        train_preds, train_labels = [], []
        
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            # Forward pass - only need x and edge_index
            out = model(batch.x, batch.edge_index)
            
            # Get predictions
            _, pred = out.max(dim=1)
            
            # Store for metrics
            train_preds.append(pred.cpu())
            train_labels.append(batch.y.cpu())
            
            # Loss and backward
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Validation
        model.eval()
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                _, pred = out.max(dim=1)
                
                val_preds.append(pred.cpu())
                val_labels.append(batch.y.cpu())
        
        # Calculate metrics
        from sklearn.metrics import precision_score, recall_score, f1_score
        
        train_preds = torch.cat(train_preds)
        train_labels = torch.cat(train_labels)
        train_prec = precision_score(train_labels, train_preds, zero_division=0)
        train_rec = recall_score(train_labels, train_preds, zero_division=0)
        train_f1 = f1_score(train_labels, train_preds, zero_division=0)
        
        val_preds = torch.cat(val_preds)
        val_labels = torch.cat(val_labels)
        val_prec = precision_score(val_labels, val_preds, zero_division=0)
        val_rec = recall_score(val_labels, val_preds, zero_division=0)
        val_f1 = f1_score(val_labels, val_preds, zero_division=0)
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train: Loss={total_loss/len(train_loader):.4f}, F1={train_f1:.4f}")
        print(f"  Precision={train_prec:.4f}, Recall={train_rec:.4f}")
        print(f"  Val: F1={val_f1:.4f}, Precision={val_prec:.4f}, Recall={val_rec:.4f}")
    
    # Save final model
    torch.save(model.state_dict(), "simple_path_model.pt")
    
    return model

In [122]:
def predict_simple_path(model, G, node1, node2):
    """Predict nodes on the path between two nodes"""
    model.eval()
    device = next(model.parameters()).device
    
    # Get node indices
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Check nodes exist
    if node1 not in node_to_idx or node2 not in node_to_idx:
        print("Error: One or both nodes not found in graph")
        return []
    
    # Create edge index
    edge_index = []
    for u, v in G.edges():
        edge_index.append([node_to_idx[u], node_to_idx[v]])
        edge_index.append([node_to_idx[v], node_to_idx[u]])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().to(device)
    
    # Create input features (just marking the two input nodes)
    x = torch.zeros((len(node_list), 2), dtype=torch.float).to(device)
    x[node_to_idx[node1], 0] = 1.0
    x[node_to_idx[node2], 1] = 1.0
    
    # Make prediction
    with torch.no_grad():
        out = model(x, edge_index)
        probs = torch.softmax(out, dim=1)
        _, pred = out.max(dim=1)
    
    # Get nodes predicted to be on the path
    path_indices = torch.nonzero(pred == 1).squeeze().cpu().tolist()
    
    # Handle case of single or no result
    if not isinstance(path_indices, list):
        path_indices = [path_indices] if torch.is_tensor(path_indices) and path_indices.numel() > 0 else []
    
    # Convert to node names
    path_nodes = [node_list[idx] for idx in path_indices 
                 if idx != node_to_idx[node1] and idx != node_to_idx[node2]]
    
    return path_nodes, out.cpu(), node_list

In [141]:
simple_model = train_simple_path_model(G_nx, num_epochs=50)


Creating path samples...
Generated 2000 samples from 31 unique rings
Training model...
Epoch 1/50:
  Train: Loss=0.5046, F1=0.1784
  Precision=0.2711, Recall=0.1330
  Val: F1=0.3817, Precision=0.2847, Recall=0.5791
Epoch 2/50:
  Train: Loss=0.0609, F1=0.4174
  Precision=0.2813, Recall=0.8083
  Val: F1=0.4096, Precision=0.2618, Recall=0.9411
Epoch 3/50:
  Train: Loss=0.0336, F1=0.4043
  Precision=0.2558, Recall=0.9637
  Val: F1=0.4183, Precision=0.2660, Recall=0.9779
Epoch 4/50:
  Train: Loss=0.0307, F1=0.4107
  Precision=0.2601, Recall=0.9750
  Val: F1=0.4291, Precision=0.2757, Recall=0.9669
Epoch 5/50:
  Train: Loss=0.0268, F1=0.4646
  Precision=0.3059, Recall=0.9659
  Val: F1=0.5460, Precision=0.3854, Recall=0.9362
Epoch 6/50:
  Train: Loss=0.0225, F1=0.6198
  Precision=0.4615, Recall=0.9435
  Val: F1=0.6657, Precision=0.5332, Recall=0.8859
Epoch 7/50:
  Train: Loss=0.0196, F1=0.6590
  Precision=0.5124, Recall=0.9232
  Val: F1=0.6743, Precision=0.5520, Recall=0.8663
Epoch 8/50:
  Tra

In [142]:
node1 = "NODE_4_0_1"
node2 = "NODE_4_0_4"
path_nodes, out, node_list = predict_simple_path(simple_model, G_nx, node1, node2)

print(f"Isolated nodes because of {node1} and {node2}:")
for node in path_nodes:
    print(f"- {node}")

Isolated nodes because of NODE_4_0_1 and NODE_4_0_4:
- NODE_4_0_2
- NODE_4_0_3
- NODE_4_0_5
